# CNN based Rainbow agent against fixed opponent

In the previous notebook, `9-rainbow.ipynb`, we created a CNN based Rainbow agent and found the results to be quite good.
However, agent 2 was significantly worse in human play than agent 1.
As an attempt to resolve this, we let the rainbow agents train further by playing against a frozen opponent, forming somewhat of a "league based" training cycle.
This is based on the approach used in `7-cnn-dqn-fixed-opponent.ipynb` and `8-mlp-dqn-fixed-opponent.ipynb`.

<hr><hr>

## Table of Contents

- Contact information
- Checking requirements
  - Correct Anaconda environment
  - Correct module access
  - Correct CUDA access
- Training rainbow agent against frozen rainbow agent
  - Building the environment
  - Implementing the Rainbow policy
  - Building agents
  - Function for letting agents learn
  - Function for watching learned agent
  - Doing the experiment
- Discussion

<hr><hr>

## Contact information

| Name             | Student ID | VUB mail                                                  | Personal mail                                               |
| ---------------- | ---------- | --------------------------------------------------------- | ----------------------------------------------------------- |
| Lennert Bontinck | 0568702    | [lennert.bontinck@vub.be](mailto:lennert.bontinck@vub.be) | [info@lennertbontinck.com](mailto:info@lennertbontinck.com) |



<hr><hr>

## Checking requirements

### Correct Anaconda environment

The `rl-project` anaconda environment should be active to ensure proper support. Installation instructions are available on [the GitHub repository of the RL course project and homeworks](https://github.com/pikawika/vub-rl).

In [1]:
####################################################
# CHECKING FOR RIGHT ANACONDA ENVIRONMENT
####################################################

import os
from platform import python_version

print(f"Active environment: {os.environ['CONDA_DEFAULT_ENV']}")
print(f"Correct environment: {os.environ['CONDA_DEFAULT_ENV'] == 'rl-project'}")
print(f"\nPython version: {python_version()}")
print(f"Correct Python version: {python_version() == '3.8.10'}")

Active environment: rl-project
Correct environment: True

Python version: 3.8.10
Correct Python version: True


<hr>

### Correct module access

The following code block will load in all required modules and show if the versions match those that are recommended.

In [3]:
####################################################
# LOADING MODULES
####################################################

# Allow reloading of libraries
import importlib

# Plotting
import matplotlib; print(f"Matplotlib version (3.5.1 recommended): {matplotlib.__version__}")
import matplotlib.pyplot as plt

# Argparser
import argparse

# More data types
import typing
import numpy as np

# Pygame
import pygame; print(f"Pygame version (2.1.2 recommended): {pygame.__version__}")

# Gym environment
import gym; print(f"Gym version (0.21.0 recommended): {gym.__version__}")

# Tianshou for RL algorithms
import tianshou as ts; print(f"Tianshou version (0.4.8 recommended): {ts.__version__}")

# Torch is a popular DL framework
import torch; print(f"Torch version (1.12.0 recommended): {torch.__version__}")

# PPrint is a pretty print for variables
from pprint import pprint

# Our custom connect four gym environment
import sys
sys.path.append('../')
import gym_connect4_pygame.envs.ConnectFourPygameEnvV2 as cfgym
importlib.invalidate_caches()
importlib.reload(cfgym)

# Time for allowing "freezes" in execution
import time;

# Allow for copying objects in a non reference manner
import copy

# Used for updating notebook display
from IPython.display import clear_output

Matplotlib version (3.5.1 recommended): 3.5.1
Pygame version (2.1.2 recommended): 2.1.2
Gym version (0.21.0 recommended): 0.21.0
Tianshou version (0.4.8 recommended): 0.4.8
Torch version (1.12.0 recommended): 1.12.0.dev20220520+cu116


<hr>

### Correct CUDA access

The installation instructions specify how to install PyTorch with CUDA 11.6.
The following code block tests if this was done successfully.

In [4]:
####################################################
# CUDA VALIDATION
####################################################

# Check cuda available
print(f"CUDA is available: {torch.cuda.is_available()}")

# Show cuda devices
print(f"\nAmount of connected devices supporting CUDA: {torch.cuda.device_count()}")

# Show current cuda device
print(f"\nCurrent CUDA device: {torch.cuda.current_device()}")

# Show cuda device name
print(f"Cuda device 0 name: {torch.cuda.get_device_name(0)}")

CUDA is available: True

Amount of connected devices supporting CUDA: 1

Current CUDA device: 0
Cuda device 0 name: NVIDIA GeForce GTX 970


<hr><hr>

## Training rainbow agent against frozen rainbow agent

Our connect four gym setup requires two agents, one for each player.
To reduce complexity, agents will always play as the same player, e.g. always as player 1.
It is important to note that connect four is a *solved game*.
According to [The Washington Post](https://www.washingtonpost.com/news/wonk/wp/2015/05/08/how-to-win-any-popular-game-according-to-data-scientists/):

> Connect Four is what mathematicians call a "solved game," meaning you can play it perfectly every time, no matter what your opponent does. You will need to get the first move, but as long as you do so, you can always win within 41 moves.

<hr>

### Building the environment

This code is identical to the notebook `9-rainbow.ipynb`, a reward for blocking moves is given.

In [5]:
####################################################
# CONNECT FOUR V2 ENVIRONMENT
####################################################

def get_env():
    """
    Returns the connect four gym environment V2 altered for Tianshou and Petting Zoo compatibility.
    Already wrapped with a ts.env.PettingZooEnv wrapper.
    """
    return ts.env.PettingZooEnv(cfgym.env(reward_move= 0, # Set to 1 for reward to make moves (incentivise longer games)
                                          reward_blocking= 1, # Set to 1 for reward to make blocking moves (incentivise defensive games)
                                          reward_invalid= -3,
                                          reward_draw= 3,
                                          reward_win= 5,
                                          reward_loss= -5,
                                          allow_invalid_move= False))
    
    
# Test the environment
env = get_env()
print(f"Observation space: {env.observation_space}")
print(f"\nAction space: {env.action_space}")

# Reset the environment to start from a clean state, returns the initial observation
observation = env.reset()

print("\n Initial player id:")
print(observation["agent_id"])

print("\n Initial observation:")
print(observation["obs"])

print("\n Initial mask:")
print(observation["mask"])

# Clean unused variables
del observation
del env

Observation space: Dict(action_mask:Box([0 0 0 0 0 0 0], [1 1 1 1 1 1 1], (7,), int8), observation:Box([[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]], [[2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]], (6, 7), int8))

Action space: Discrete(7)

 Initial player id:
player_1

 Initial observation:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]

 Initial mask:
[True, True, True, True, True, True, True]


In [6]:
####################################################
# BLOCKING MOVE CHECK
####################################################

# Check if a reward is received for playing a blocking move

env = get_env()
env.reset()
env.step(action= 0)
print(env.rewards)
env.step(action= 1)
print(env.rewards)
env.step(action= 0)
print(env.rewards)
env.step(action= 1)
print(env.rewards)
env.step(action= 0)
print(env.rewards)
env.step(action= 1)
print(env.rewards)
env.step(action= 1)
print(f"Blocking move made by player 1: {env.rewards}")

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
Blocking move made by player 1: [1, 0]


<hr>

### Implementing the Rainbow policy

This code is identical to the notebook `9-rainbow.ipynb`, the defaults are changed so that they reflect the best found parameters.

In [7]:
####################################################
# DQN ARCHITECTURE
####################################################

class CNNForRainbow(torch.nn.Module):
    """
    Custom CNN to be used as baseclass for the Rainbow algorithm.
    Extracts "feautures" for the Rainbow algorithm by doing a 4x4 cnn kernel pass and providing 64 filters for each mask.
    """
    def __init__(self,
                 state_shape: typing.Sequence[int],
                 device: typing.Union[str, int, torch.device] = 'cuda' if torch.cuda.is_available() else 'cpu'):
        
        # Torch init
        super().__init__()
        
        # Store device to be used
        self.device = device
        
        # The input layer is singular -> we have 1 board vector
        input_channels_cnn = 1
        
        # We output 64 filters per kernel 
        output_channels_cnn = 64 # Updated from previous 16
        
        # We store the output dimension of the CNN "feature" layer
        self.output_dim = (state_shape[0] - 3) * (state_shape[1] - 3) * output_channels_cnn
        
        self.net = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels= input_channels_cnn, out_channels= output_channels_cnn, kernel_size= 4, stride= 1), torch.nn.ReLU(inplace=True),
            torch.nn.Flatten(),
        )

    def forward(self,
                obs: typing.Union[np.ndarray, torch.Tensor],
                state: typing.Optional[typing.Any] = None,
                info: typing.Dict[str, typing.Any] = {}):
        # Make a torch instance (from regular vector of board)
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float, device=self.device)
            
        # Tianshou bugs the batch output, reshape to work properly with our torch version
        if (len(np.shape(obs)) != 4):
            obs = obs[:, None, :, :]
        
        # Return what is needed (network output & state)
        return self.net(obs), state


In [8]:
####################################################
# RAINBOW ARCHITECTURE
####################################################

class Rainbow(CNNForRainbow):
    """
    Implementation of the Rainbow algorithm making using of the CNNForRainbow baseclass.
    Default parameters adopted from: https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py
    """

    def __init__(self,
                 state_shape: typing.Sequence[int],
                 action_shape: typing.Sequence[int],
                 device: typing.Union[str, int, torch.device] = 'cuda' if torch.cuda.is_available() else 'cpu',
                 num_atoms: int = 51,
                 is_noisy: bool = True,
                 noisy_std: float = 0.1,
                 is_dueling: bool = True):
        
        # Init CNN feature extraction parent class
        super().__init__(state_shape= state_shape, device= device)
        
        # the amount of actions we have is just the action shape
        self.action_num = np.prod(action_shape)
        
        # Store class specific info
        self.num_atoms = num_atoms
        self._is_dueling = is_dueling

        # Our linear layer depends on wether or not we want to use a noisy environment
        # Noisy implementation based on https://arxiv.org/abs/1706.10295
        def linear(x, y):
            if is_noisy:
                return ts.utils.net.discrete.NoisyLinear(x, y, noisy_std)
            else:
                return torch.nn.Linear(x, y)
            
        # Specify Q and V based on wether or not agent is dueling
        # Setting agent on dueling mode should help generalisation according to rainbow paper
        # NOTE: this uses the output dim from the feature extraction CNN
        self.Q = torch.nn.Sequential(
            linear(self.output_dim, 512), torch.nn.ReLU(inplace=True),
            linear(512, self.action_num * self.num_atoms))
        
        if self._is_dueling:
            self.V = torch.nn.Sequential(
                linear(self.output_dim, 512), torch.nn.ReLU(inplace=True),
                linear(512, self.num_atoms))
            
        # New output dim for this rainbow network
        self.output_dim = self.action_num * self.num_atoms
        

    def forward(self,
                obs: typing.Union[np.ndarray, torch.Tensor],
                state: typing.Optional[typing.Any] = None,
                info: typing.Dict[str, typing.Any] = {}):
        
        # Use our parent CNN based network to get "features"
        obs, state = super().forward(obs)
        
        # Get our Rainbow specific values
        q = self.Q(obs)
        q = q.view(-1, self.action_num, self.num_atoms)
        
        if self._is_dueling:
            v = self.V(obs)
            v = v.view(-1, 1, self.num_atoms)
            logits = q - q.mean(dim=1, keepdim=True) + v
        else:
            logits = q
        
        # We need to go from our logits to an accepted dimension of probability outputs
        probs = logits.softmax(dim=2)
        
        return probs, state

In [9]:
####################################################
# RAINBOW POLICY
####################################################

def rainbow_policy(state_shape: tuple,
                   action_shape: tuple,
                   optim: typing.Optional[torch.optim.Optimizer] = None,
                   learning_rate: float =  0.0001, # Increased from 0000625
                   gamma: float = 0.8, # Decreased from 0.9
                   n_step: int = 3,
                   num_atoms: int = 51,
                   is_noisy: bool = True,
                   noisy_std: float = 0.1,
                   is_dueling: bool = True,
                   frozen: bool = False, # Added to freeze an agent
                   target_update_freq: int = 500):
    """
    Implementation of the Rainbow policy.
    Default parameters adopted from: https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py
    """
    
    # Use cuda device if possible
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Rainbow network to be used by policy
    net = Rainbow(state_shape= state_shape,
                  action_shape= action_shape,
                  device= device,
                  num_atoms= num_atoms,
                  is_noisy= is_noisy,
                  noisy_std= noisy_std,
                  is_dueling= is_dueling).to(device)
    
    # Default optimizer is an adam optimizer with the argparser learning rate
    if optim is None:
        optim = torch.optim.Adam(net.parameters(), lr= learning_rate)
        
    # If we are frozen, we use an optimizer that has learning rate 0
    if frozen:
        optim = torch.optim.SGD(net.parameters(), lr= 0)
        
    # Our agents Rainbow policy
    return ts.policy.RainbowPolicy(model= net,
                                   optim= optim,
                                   discount_factor= gamma,
                                   num_atoms= num_atoms,
                                   estimation_step= n_step,
                                   target_update_freq= target_update_freq).to(device)
    
    

<hr>

### Building agents

This code is identical to the notebook `9-rainbow.ipynb`, with the added option of "freezing" an agent which corresponds to giving it an optimizer with learning rate 0.

In [10]:
####################################################
# AGENT CREATION
####################################################

def get_agent_manager(agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
                      agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
                      agent_player1_frozen: bool = False, # Freeze a player -> don't let it learn further
                      agent_player2_frozen: bool = False,
                      optim: typing.Optional[torch.optim.Optimizer] = None):
    """
    Gets a multi agent policy manager, optimizer and player ids for the connect four V2 gym environment.
    Per default this returns 
        - Multi agent manager for 2 agents using Rainbow
        - Adam optimizer
        - ['player_1', 'player_2'] from the connect four environment
    """
    
    # Get the environment to play in (Connect four gym V2)
    env = get_env()
    
    # Get the observation space from the environment, depending on typo of space (ternary operator)
    observation_space = env.observation_space['observation'] if isinstance(env.observation_space, gym.spaces.Dict) else env.observation_space
    
    # Set the arguments
    state_shape = observation_space.shape or observation_space.n
    action_shape = env.action_space.shape or env.action_space.n
    
    # Configure agent player 1 to be a Rainbow if no policy is passed.
    if agent_player1 is None:
        # Our agent1 uses a Rainbow policy
        agent_player1 = rainbow_policy(state_shape= state_shape,
                                       action_shape= action_shape,
                                       optim= optim,
                                       frozen= agent_player1_frozen)
    
    # Configure agent player 2 to be a Rainbow if no policy is passed.
    if agent_player2 is None:
        # Our agent1 uses a Rainbow policy
        agent_player2 = rainbow_policy(state_shape= state_shape,
                                       action_shape= action_shape,
                                       optim= optim,
                                       frozen= agent_player2_frozen)

    # Default order of the agents
    agents = [agent_player1, agent_player2]
        
    # Create the multi agent policy
    policy = ts.policy.MultiAgentPolicyManager(agents, env)
    
    # Return our policy, optimizer and the available agents in the environment
    # Per default: 
    #   - Multi agent manager for 2 agents using Rainbow
    #   - Adam optimizer
    #   - ['player_1', 'player_2'] from the connect four environment
    
    return policy, optim, env.agents

<hr>

### Function for letting agents learn

This code is identical to the notebook `9-rainbow.ipynb`, but a stopping condition is added and the defaults are updated to the newly found best, the reward metric is also updated to relfect the score of the non frozen agent.
The testing strategy is also updated to be on one environment using 10 trials.
We also decay the epsilon faster and don't use epsilon decay on the frozen agent.

In [15]:
####################################################
# AGENT TRAINING
####################################################

def train_agent(filename: str,
                agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
                agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
                agent_player1_frozen: bool = False, # Freeze a player -> don't let it learn further
                agent_player2_frozen: bool = False,
                single_agent_score_as_reward: bool= False, # Uses non frozen agent's score as reward
                optim: typing.Optional[torch.optim.Optimizer] = None,
                training_env_num: int = 10,
                testing_env_num: int = 10,
                episode_per_test: int = 10,
                stopping_threshold: float = 7,
                buffer_size: int = 10000, # Default 100000
                batch_size: int = 64, # Default 32
                epochs: int = 500, # Default 50
                step_per_epoch: int = 10000,
                step_per_collect: int = 10, # Should be multiple of the test/training envs
                update_per_step: float = 0.1,
                testing_eps: float = 0.005,
                training_eps_init: float = 1,
                training_eps_final: float = 0.2): # Default 0.05
    """
    Trains two agents in the connect four V2 environment and saves their best model and logs.
    Returns:
        - result from offpolicy_trainer
        - final version of agent 1
        - final version of agent 2
    Defaults adopted from: https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py
    """

    # ======== notebook specific =========
    notebook_version = '10' # Used for foldering logs and models

    # ======== environment setup =========
    train_envs = ts.env.DummyVectorEnv([get_env for _ in range(training_env_num)])
    test_envs = ts.env.DummyVectorEnv([get_env for _ in range(testing_env_num)])
    
    # set the seed for reproducibility
    np.random.seed(1998)
    torch.manual_seed(1998)
    train_envs.seed(1998)
    test_envs.seed(1998)

    # ======== agent setup =========
    # Gets our agents from the previously made function
    # Per default: 
    #   - Multi agent manager for 2 agents using Rainbow
    #   - Adam optimizer
    #   - ['player_1', 'player_2'] from the connect four environment
    policy, optim, agents = get_agent_manager(agent_player1=agent_player1,
                                              agent_player2=agent_player2,
                                              agent_player1_frozen= agent_player1_frozen,
                                              agent_player2_frozen= agent_player2_frozen,
                                              optim=optim)

    # ======== collector setup =========
    # Make a collector for the training environments
    buffer= ts.data.VectorReplayBuffer(total_size= buffer_size,
                                       buffer_num=len(train_envs))
    
    train_collector = ts.data.Collector(policy= policy,
                                        env= train_envs,
                                        buffer= buffer,
                                        exploration_noise= True)
    
    # Make a collector for the testing environments
    test_collector = ts.data.Collector(policy= policy,
                                       env= test_envs,
                                       exploration_noise= True)
    
    # ======== ensure folders exist =========
    if not os.path.exists(os.path.join('./logs', 'paper_notebooks', notebook_version, filename)):
        os.makedirs(os.path.join('./logs', 'paper_notebooks', notebook_version, filename))
    if not os.path.exists(os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename)):
        os.makedirs(os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename))

    # ======== tensorboard logging setup =========
    # Allows to save the training progress to tensorboard compatable logs
    log_path = os.path.join('./logs', 'paper_notebooks', notebook_version, filename)
    writer = torch.utils.tensorboard.SummaryWriter(log_path)
    logger = ts.utils.TensorboardLogger(writer)

    # ======== callback functions used during training =========
    # We want to save our best policy
    def save_best_fn(policy):
        """
        Callback to save the best model
        """
        # Save best agent 1
        model_save_path = os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename, 'best_policy_agent1.pth')
        torch.save(policy.policies[agents[0]].state_dict(), model_save_path)
        
        # Save best agent 2
        model_save_path = os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename, 'best_policy_agent2.pth')
        torch.save(policy.policies[agents[1]].state_dict(), model_save_path)
        
        # Save agent2

    def stop_fn(average_rews):
        """
        Callback to stop training when we've reached the desired reward.
        Reward is the test average return value of the reward_metric function.
        """
        if single_agent_score_as_reward:
            # Get singular episode mean reward
            episode_reward= average_rews / episode_per_test
            stop= episode_reward >= stopping_threshold
            print(f"testing for stop: {episode_reward} >= {stopping_threshold} -> {stop}")
            # Agent is seen as "trained enough"
            return stop
        else:
            return False # Not implemented

    def train_fn(epoch, env_step):
        """
        Callback before training, sets the training epsilon in a decaying manner.
        Adopted from: https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py
        """        
        # Nature DQN setting to have a "linear decaying epsilon" for the first 50 thousand iterations
        if env_step <= 50000:
            training_eps = training_eps_init - env_step / 1000000 * (training_eps_init - training_eps_final)
        else:
            training_eps = training_eps_final
            
            
        # Set epsilon
        policy.policies[agents[0]].set_eps(training_eps)
        policy.policies[agents[1]].set_eps(training_eps)
        
        # If frozen we don't have a large epsilon
        if agent_player1_frozen:
            policy.policies[agents[0]].set_eps(training_eps_final)
        if agent_player2_frozen:
            policy.policies[agents[1]].set_eps(training_eps_final)

    def test_fn(epoch, env_step):
        """
        Callback beore testing, sets the testing epsilon.
        """        
        # Before testing we want to configure the epsilon for the agents
        # In general more greedy than the train case but not
        #   to avoid getting stuck on invalid moves
        policy.policies[agents[0]].set_eps(testing_eps)
        policy.policies[agents[1]].set_eps(testing_eps)

    def reward_metric(rews):
        """
        Callback for reward collection.
        Currently the reward is the sum of both agents.
        """        
        if agent_player2_frozen and single_agent_score_as_reward:
            # agent 2 frozen, optimizing for agent 1
            return rews[:, 0]
        
        if agent_player1_frozen and single_agent_score_as_reward:
            # agent 1 frozen, optimizing for agent 2
            return rews[:, 1]
        
        # Per default we are interested in optimizing both agents
        return rews[:, 0] + rews[:, 1]

    # ======== Training =========
    # off policy training
    result = ts.trainer.offpolicy_trainer(policy= policy,
                                          train_collector= train_collector,
                                          test_collector= test_collector,
                                          max_epoch= epochs,
                                          step_per_epoch= step_per_epoch,
                                          step_per_collect= step_per_collect,
                                          episode_per_test= episode_per_test,
                                          batch_size= batch_size,
                                          train_fn= train_fn,
                                          test_fn= test_fn,
                                          stop_fn= stop_fn,
                                          save_best_fn= save_best_fn,
                                          update_per_step= update_per_step,
                                          logger= logger,
                                          test_in_train= False,
                                          reward_metric= reward_metric)
    
    # Save final agent 1
    model_save_path = os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename, 'final_policy_agent1.pth')
    torch.save(policy.policies[agents[0]].state_dict(), model_save_path)

    # Save final agent 2
    model_save_path = os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename, 'final_policy_agent2.pth')
    torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

    return result, policy.policies[agents[0]], policy.policies[agents[1]]


<hr>

### Function for watching learned agent

Identical to the previous notebook.

In [21]:
####################################################
# WATCHING THE LEARNED POLICY IN ACTION
####################################################

def watch(numer_of_games: int = 3,
          agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
          agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
          test_epsilon: float = 0.005, # For the watching we act completely greedy but low random for not getting stuck on invalid move
          render_speed: float = 0.15, # Amount of seconds to update frame/ do a step
          ) -> None:
    
    # Get the connect four V2 environment (must be a list)
    env= ts.env.DummyVectorEnv([get_env])
    
    # Get the agents from the trained agents
    policy, optim, agents = get_agent_manager(agent_player1= agent_player1,
                                              agent_player2= agent_player2)
    
    # Evaluate the policy
    policy.eval()
    
    # Set the testing policy epsilon for our agents
    policy.policies[agents[0]].set_eps(test_epsilon)
    policy.policies[agents[1]].set_eps(test_epsilon)
    
    # Collect the test data
    collector = ts.data.Collector(policy= policy,
                                  env= env,
                                  exploration_noise= True)
    
    # Render games in human mode to see how it plays
    result = collector.collect(n_episode= numer_of_games, render= render_speed)
    
    # Close the environment aftering collecting the results
    # This closes the pygame window after completion
    env.close()
    
    # Get the rewards and length from the test trials
    rewards, length = result["rews"], result["lens"]
    
    # Print the final reward for the first agent
    print(f"Average steps of game:  {length.mean()}")
    print(f"Final mean reward agent 1: {rewards[:, 0].mean()}, std: {rewards[:, 0].std()}")
    print(f"Final mean reward agent 2: {rewards[:, 1].mean()}, std: {rewards[:, 1].std()}")

<hr>

### Doing the experiment

To test if we can better train our agents, we train them against a frozen oponent until the stopping criteria is reached or 50 epochs are passed.
We loop 20 times with the previously found best parameters, optimizing each agent 10 times against a frozen one.
We store the output of each experiment but only save a select few to GitHub due to data saving reasons.

1. Experiment 1
   - Starting params agent 1: `./saved_variables/paper_notebooks/9/rainbow_vs_rainbow_blocking_reward_complex_cnn/best_policy_agent1.pth`
   - Starting params agent 2: `./saved_variables/paper_notebooks/9/rainbow_vs_rainbow_blocking_reward_complex_cnn/final_policy_agent2.pth`
   - filename: `1-50epoch_20loop/looping-iteration-i`
   - epochs: `50`
   - loops: `20`
   - others: `default`
   - stopping threshold: `10`


In [19]:
####################################################
# EXPERIMENT: TRAINING AGENTS
####################################################

# Initial start for agent 1
agent1_starting_params = "./saved_variables/paper_notebooks/9/rainbow_vs_rainbow_blocking_reward_complex_cnn/best_policy_agent1.pth"

# Initial start for agent 2
agent2_starting_params = "./saved_variables/paper_notebooks/9/rainbow_vs_rainbow_blocking_reward_complex_cnn/final_policy_agent2.pth"

# Experiment settings
epochs = 50
loops = 20
stopping_threshold = 10

# Filename prefix
filename_prefix = "1-50epoch_20loop/looping-iteration-"

for loop_idx in range(loops):
    # Filename
    filename = filename_prefix + str(loop_idx)
    
    # Use provided starting params in first loop, the one from previous iteration in next
    if loop_idx > 0:
        agent1_starting_params = "./saved_variables/paper_notebooks/10/" + filename_prefix + str(loop_idx - 1) + "/best_policy_agent1.pth"
        agent2_starting_params = "./saved_variables/paper_notebooks/10/" + filename_prefix + str(loop_idx - 1) + "/best_policy_agent2.pth"
    
    # Determine what agent to freeze
    freeze_agent1 = True if loop_idx % 2 == 1 else False
    freeze_agent2 = True if loop_idx % 2 == 0 else False
    
    # Show info
    print()
    training_agent = "1" if freeze_agent2 else "2"
    print(f"Started training agent player {training_agent} with frozen oponent in loop {loop_idx}")
    
    # Get the environment settings
    env = get_env()
    observation_space = env.observation_space['observation'] if isinstance(env.observation_space, gym.spaces.Dict) else env.observation_space
    state_shape = observation_space.shape or observation_space.n
    action_shape = env.action_space.shape or env.action_space.n
    
    # Configure agent 1
    agent1 = rainbow_policy(state_shape= state_shape,
                            action_shape= action_shape)
    
    if agent1_starting_params:
        agent1.load_state_dict(torch.load(agent1_starting_params))
        
    # Configure agent 2
    agent2 = rainbow_policy(state_shape= state_shape,
                            action_shape= action_shape)
    
    if agent2_starting_params:
        agent2.load_state_dict(torch.load(agent2_starting_params))
        
        
    # Train the agent
    off_policy_traininer_results, final_agent_player1, final_agent_player2 = train_agent(epochs= epochs,
                                                                                         agent_player1= agent1,
                                                                                         agent_player1_frozen = freeze_agent1,
                                                                                         agent_player2= agent2,
                                                                                         agent_player2_frozen = freeze_agent2,
                                                                                         filename= filename,
                                                                                         stopping_threshold= stopping_threshold,
                                                                                         single_agent_score_as_reward = True)
            
            


Started training agent player 1 with frozen oponent in loop 0


Epoch #1: 10001it [00:33, 300.42it/s, env_step=10000, len=10, n/ep=0, n/st=10, player_1/loss=1.456, player_2/loss=1.521, rew=-3.00]


Epoch #1: test_reward: 44.700000 ± 18.633572, best_reward: 44.700000 ± 18.633572 in #1
testing for stop: 4.470000000000001 >= 10 -> False


Epoch #2: 10001it [00:32, 309.16it/s, env_step=20000, len=12, n/ep=2, n/st=10, player_1/loss=1.423, player_2/loss=1.379, rew=0.00]


Epoch #2: test_reward: 26.000000 ± 0.000000, best_reward: 44.700000 ± 18.633572 in #1
testing for stop: 4.470000000000001 >= 10 -> False


...

Epoch #1: test_reward: -2.000000 ± 2.408319, best_reward: 103.900000 ± 44.227706 in #0
testing for stop: 10.39 >= 10 -> True

Started training agent player 1 with frozen oponent in loop 18


Epoch #1: 10001it [00:30, 326.70it/s, env_step=10000, len=10, n/ep=0, n/st=10, player_1/loss=1.338, player_2/loss=1.265, rew=-5.00]


Epoch #1: test_reward: 20.600000 ± 8.616264, best_reward: 118.600000 ± 52.830294 in #0
testing for stop: 11.86 >= 10 -> True

Started training agent player 2 with frozen oponent in loop 19


Epoch #1: 10001it [00:30, 327.71it/s, env_step=10000, len=11, n/ep=1, n/st=10, player_1/loss=1.146, player_2/loss=1.460, rew=-5.00]


Epoch #1: test_reward: -2.000000 ± 2.408319, best_reward: 103.900000 ± 44.227706 in #0
testing for stop: 10.39 >= 10 -> True


In [22]:
####################################################
# EXPERIMENT: VIEWING THE BEST LEARNED POLICY
####################################################

# Get the environment settings
env = get_env()
observation_space = env.observation_space['observation'] if isinstance(env.observation_space, gym.spaces.Dict) else env.observation_space
state_shape = observation_space.shape or observation_space.n
action_shape = env.action_space.shape or env.action_space.n

# Configure the best agent
best_agent1 = rainbow_policy(state_shape= state_shape,
                             action_shape= action_shape)
best_agent1.load_state_dict(torch.load("./saved_variables/paper_notebooks/10/1-50epoch_20loop/looping-iteration-18/best_policy_agent1.pth"))
best_agent1.set_eps(0)


best_agent2 = rainbow_policy(state_shape= state_shape,
                             action_shape= action_shape)
best_agent2.load_state_dict(torch.load("./saved_variables/paper_notebooks/10/1-50epoch_20loop/looping-iteration-19/best_policy_agent2.pth"))
best_agent2.set_eps(0)

# Watch the best agent at work
watch(numer_of_games= 3,
      render_speed= 0.3,
      agent_player1= best_agent1,
      agent_player2= best_agent2)



Average steps of game:  36.0
Final mean reward agent 1: 106.33333333333333, std: 54.68292441175967
Final mean reward agent 2: 95.33333333333333, std: 43.36921591277491


<hr><hr>

## Discussion

It becomes clear that the agent starts to overfit to the oponent agent rather quickly, the agent knows the rules significantly enough to outsmart a stationary oponent agent in very few training loops.
A different testing strategy is needed to have it learn connect four better rather then have it learn the oponent better.


In [23]:
####################################################
# CLEAN VARIABLES
####################################################

del action_shape
del agent1
del agent1_starting_params
del agent2
del agent2_starting_params
del env
del epochs
del filename
del filename_prefix
del final_agent_player1
del final_agent_player2
del freeze_agent1
del freeze_agent2
del loop_idx
del loops
del observation_space
del off_policy_traininer_results
del state_shape
